In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()

In [68]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [69]:
image_datasets = {x: datasets.ImageFolder(os.path.join('.', x),  data_transforms[x]) for x in ['train', 'val']}

In [70]:
image_datasets

{'train': Dataset ImageFolder
     Number of datapoints: 6151
     Root Location: ./train
     Transforms (if any): Compose(
                              ToTensor()
                              Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                          )
     Target Transforms (if any): None, 'val': Dataset ImageFolder
     Number of datapoints: 1334
     Root Location: ./val
     Transforms (if any): Compose(
                              Resize(size=256, interpolation=PIL.Image.BILINEAR)
                              CenterCrop(size=(224, 224))
                              ToTensor()
                              Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                          )
     Target Transforms (if any): None}

In [71]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, **kwargs) for x in ['train', 'val']}

In [72]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

In [73]:
class_names = image_datasets['train'].classes

In [74]:
class_names

['square', 'triangle']

In [75]:
 use_cuda = torch.cuda.is_available()

In [76]:
use_cuda

True

In [77]:
device = torch.device("cuda" if use_cuda else "cpu")

In [78]:
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [103]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(47, 3760)
        self.fc2 = nn.Linear(3760, 2)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        # x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [104]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, size_average=False).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(test_loader.dataset),100. * correct / len(test_loader.dataset)))

In [105]:
model = Net().to(device)

In [106]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [107]:
args= {'log_interval': 10}

In [108]:
dataloaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x7f445efbb358>,
 'val': <torch.utils.data.dataloader.DataLoader at 0x7f445ef1e8d0>}

In [109]:
import torch.nn.functional as F

In [110]:
for epoch in range(1, 4):
    train(args, model, device, dataloaders["train"], optimizer, epoch)
    test(args, model, device, dataloaders["val"])

RuntimeError: invalid argument 1: only batches of spatial targets supported (3D tensors) but got targets of dimension: 1 at /pytorch/aten/src/THCUNN/generic/SpatialClassNLLCriterion.cu:14

In [58]:
dataloaders["train"].dataset

('./train/triangle/1810.png', 1)